In [1]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords

In [7]:
df=pd.read_csv("spam.csv",encoding='ISO-8859-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [48]:
df['messages'][2]

"Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"

In [16]:
df=df[['v1','v2']]
df = df.rename(columns={'v2': 'messages', 'v1': 'label'})
df.head()


,label,messages
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [13]:
nltk.download("stopwords")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\NIRMAL\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [17]:
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    # convert to lowercase
    text = text.lower()
    # remove special characters
    text = re.sub(r'[^0-9a-zA-Z]', ' ', text)
    # remove extra spaces
    text = re.sub(r'\s+', ' ', text)
    # remove stopwords
    text = " ".join(word for word in text.split() if word not in STOPWORDS)
    return text

In [18]:
# clean the messages
df['clean_text'] = df['messages'].apply(clean_text)
df.head()

,label,messages,clean_text
0,ham,"Go until jurong point, crazy.. Available only ...",go jurong point crazy available bugis n great ...
1,ham,Ok lar... Joking wif u oni...,ok lar joking wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry 2 wkly comp win fa cup final tkts 2...
3,ham,U dun say so early hor... U c already then say...,u dun say early hor u c already say
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah think goes usf lives around though


In [22]:
df['label_num'] = df.label.map({'ham':0, 'spam':1})
df.head()

,label,messages,clean_text,label_num
0,ham,"Go until jurong point, crazy.. Available only ...",go jurong point crazy available bugis n great ...,0
1,ham,Ok lar... Joking wif u oni...,ok lar joking wif u oni,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry 2 wkly comp win fa cup final tkts 2...,1
3,ham,U dun say so early hor... U c already then say...,u dun say early hor u c already say,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah think goes usf lives around though,0


In [23]:
x=df['clean_text']
y=df['label_num']

In [20]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

In [28]:
from sklearn.svm import SVC
model = SVC(C=3)


In [29]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42, shuffle=True, stratify=y)
# model training
pipeline_model = Pipeline([('vect', CountVectorizer()),
                          ('tfidf', TfidfTransformer()),
                          ('clf', model)])
pipeline_model.fit(x_train, y_train)

print('Accuracy:', pipeline_model.score(x_test, y_test)*100)

#     cv_score = cross_val_score(model, X, y, cv=5)
#     print("CV Score:", np.mean(cv_score)*100)
y_pred = pipeline_model.predict(x_test)

print(classification_report(y_test, y_pred))

Accuracy: 98.27709978463747
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      1206
           1       1.00      0.87      0.93       187

    accuracy                           0.98      1393
   macro avg       0.99      0.94      0.96      1393
weighted avg       0.98      0.98      0.98      1393



In [37]:
x_test

3722                                 left already orchard
3062    hi babe jordan r u im home abroad lonely text ...
472     nothing meant money enters account bank remove...
4829    word checkmate chess comes persian phrase shah...
5371                                lol oh got friend dog
                              ...                        
2412    know u u know send chat 86688 let find 150p ms...
2326    xmas story peace xmas msg love xmas miracle je...
1224    winner u specially selected 2 receive 1000 cas...
3148                             sorry meeting call later
1481                               guy browsin compulsory
Name: clean_text, Length: 1393, dtype: object

In [31]:
import joblib
joblib.dump(pipeline_model, 'spamdetection.pkl')

['spamdetection.pkl']

In [34]:
import pickle
classify=joblib.load(open("spamdetection.pkl",'rb'))

In [41]:
classify.predict(x_test[[3722]])

0

In [42]:
classify

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', SVC(C=3))])